In [1]:
import requests
import pandas as pd
import os, time
from datetime import datetime, UTC
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()
API_URL = os.getenv("ITUNES_API_URL")

In [2]:
# Configuración inicial
CARPETA_DATOS = "../data/data_raw"
LOG_GENEROS = "generos_usados.txt"
os.makedirs(CARPETA_DATOS, exist_ok=True)

GENERO_TERMS = [
    "pop", "rock", "jazz", "classical", "rap", "hip hop", "reggaeton",
    "country", "electronic", "blues", "metal", "funk", "techno", "soul",
    "trap", "r&b", "dance", "alternative", "indie", "disco", "latin", "k-pop",
    "opera", "punk", "grunge", "dubstep", "house", "electro", "ambient",
    "gospel", "ska", "folk", "lofi", "world", "instrumental", "edm",
    "acoustic", "garage", "drill", "new age", "bossa nova", "samba", "cumbia",
    "flamenco", "bolero", "tango", "reggae", "afrobeats", "afropop",
    "soundtrack", "cinematic", "anime", "children", "holiday", "christmas",
    "karaoke", "spoken word"
]
GENEROS_POR_DIA = 5

In [3]:
# Funciones auxiliares
def cargar_generos_usados():
    if os.path.exists(LOG_GENEROS):
        with open(LOG_GENEROS, "r") as f:
            return set(line.strip() for line in f.readlines())
    return set()

def guardar_genero_usado(genre):
    with open(LOG_GENEROS, "a") as f:
        f.write(f"{genre}\n")

def buscar_itunes(term, limit=200):
    params = {
        "term": term,
        "limit": limit,
        "country": "US",
        "media": "music"
    }
    try:
        response = requests.get(API_URL, params=params)
        if response.status_code == 200:
            results = response.json().get("results", [])
            df = pd.json_normalize(results)
            if not df.empty:
                df["checked_at"] = datetime.now(UTC).isoformat()
                df["search_genre"] = term
                return df
    except Exception as e:
        print(f"❌ Error con '{term}': {e}")
    return pd.DataFrame()

In [4]:
# Ejecutar scraping
usados = cargar_generos_usados()
pendientes = [g for g in GENERO_TERMS if g not in usados][:GENEROS_POR_DIA]

dfs = []

for genero in pendientes:
    print(f"🎵 Buscando por género: '{genero}'")
    df = buscar_itunes(genero)
    if not df.empty:
        dfs.append(df)
        guardar_genero_usado(genero)
        print(f"✅ {len(df)} resultados para '{genero}'")
    else:
        print(f"⚠️ Sin resultados para '{genero}'")
    time.sleep(1)

if dfs:
    df_total = pd.concat(dfs, ignore_index=True)
    hoy = datetime.now(UTC).strftime("%Y-%m-%d")
    archivo_salida = f"{CARPETA_DATOS}/itunes_genero_{hoy}.csv"
    df_total.to_csv(archivo_salida, index=False)
    print(f"📦 Guardados {len(df_total)} registros en '{archivo_salida}'")

🎵 Buscando por género: 'pop'
⚠️ Sin resultados para 'pop'
🎵 Buscando por género: 'rock'
⚠️ Sin resultados para 'rock'
🎵 Buscando por género: 'jazz'
⚠️ Sin resultados para 'jazz'
🎵 Buscando por género: 'classical'
⚠️ Sin resultados para 'classical'
🎵 Buscando por género: 'rap'
⚠️ Sin resultados para 'rap'
